In [ ]:
import pandas as pd
import requests
import gmaps
from config import g_key
gmaps.configure(api_key=g_key)

In [ ]:
city_data_df = pd.read_csv("Resources/city_database.csv")
city_data_df.head()

In [ ]:
# User prompt for min and max temperature they would like
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

# New data for user preferred cities
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]                     

In [ ]:
#Checking nulls
preferred_cities_df.count()

# If null, drop...
clean_travel_cities = preferred_cities_df.dropna()

In [ ]:
# Hotel dataframe
clean_travel_cities.columns

hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [ ]:
# Hotel seach parameters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# API call to find hotels in cities
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [ ]:
# Dropping cities that didn't have hotels
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

# Save as csv
output_data_file = "Resources/vacation_data.csv"
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

###  Google map makers for hotels

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

locations = clean_hotel_df[["Lat", "Lng"]]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))